# Amber
## 0.0. PDB file 
The tutorial was written for Amber v.2024. 
Firstly we need to download the pdb file of the structure of interest (https://www.rcsb.org/)

In [ ]:
!wget https://files.rcsb.org/view/1KX5.pdb

We can make sure that the file is not empty by calling grep 

In [ ]:
!grep '^ATOM' '1KX5.pdb'| head -n 10

The structure can be visualised by nglview

In [ ]:
#In presence with water
import nglview as nv
a=nv.show_structure_file('1KX5.pdb')
a.add_representation(selection='water',repr_type='spacefill',color='red')
a.render_image()
a

In [ ]:
#Without water
import nglview as nv
a=nv.show_structure_file('1KX5.pdb')
a.render_image()
a

## 0.1. PDB File Preparation
Amber works with formated files, so in order to run molecular dynamics, especially the tleap part, it is necessary to modify them. Here we use MDAnalysis library, but you also can try pdb4amber command (See https://ambermd.org/tutorials/basic/tutorial9/index.php)

In [ ]:
#Rewrite each segid into different .pdb and merge them into modyfied sys.pdb 

import MDAnalysis as mda
!mkdir prep
!cd prep
pdb = mda.Universe('1KX5.pdb').select_atoms('protein or nucleic').write('prep/sys.pdb')

In [ ]:
%%bash

cd prep/

grep '^ATOM' sys.pdb > sys_mod.pdb

## 1.0. Topology File Preparation
In order to start a MD simulation you should preapre a topolopgy file from .pdb. It can be obtained by LEaP programm; firstly an operator choses a force field model and defines it in LEaP, after the programm generates two necessary files: topolgy file and coordinates file. To learn more about a content of each extension read information below https://ambermd.org/tutorials/pengfei/index.php  
Structurally all leaprc. files are scripts: in case of ff14SB - the protein force field - tleap run also requiers supplemental information from .dat file, containing 'full' forcefield parameter information, and .ff14SB with an auxiliary parameter information, which is added when any conflict from .dat is obsereved. 

In [ ]:
%%bash
module load amber24
dir $AMBERHOME/dat/leap/cmd/ #Here you can see a list of existing force fields
sed -n '80,84p' $AMBERHOME/dat/leap/cmd/leaprc.protein.ff14SB #parm10.dat is an example of parameter file

Now we are ready to obtain required files! We will simulate the complex in opc water, using different force fields for DNA and proteins (bsc1, ff19SB, respectivelly). For this we will use leapt runner:

In [ ]:
%%bash
module load amber24

tleap -h

In [ ]:
!cat scripts/creating_box/wo_ions.rc #the content of .rc script file

Here the size of the box is regulated in solvateoct, ion aditions are written in addions section.
Now let's run force field without setting ion concentrations:

In [ ]:
%%bash

mkdir force_field
cd force_field
mkdir box_wo_kcl
cd ..

module load amber24

tleap -s -f scripts/creating_box/wo_ions.rc # -s: programm will write a .log file; -f: file path

## 1.1. Topology file preparation: solvent parameters
However, to perform further energy minimisation it is necessary to create an electro-neutral solution, otherwise in pereodic boundary conditions (PBE) it will lead to infinitely big electrostatic interactions between the structure and other grid's boxes. Moreover it will perturbate solvent molecules near the cture and therefore will generate non-intuitive free energy changes. For better accordance with desired salt concentrarion SLTCAP method is used (DOI: 10.1021/acs.jctc.7b01254)

In [ ]:
%%bash

#Avogadro constant
n_a=6.022140857e+23 

# The density of water is ~0.997 g/mL at the room temperature, and thus, the weight of 1 L water is ~997 g.
# Since the weight of 1 mol H2O is ~18.02 g
# e.g. 1 L water is composed of ~3.33 × 10^25 H2O molecules (55.3 mol).
n_h20_in_one_liter=3.33E+25 

#Desired KCl concentration (mol/L)
n_kcl=0.15 

#The total number of water molecules
n_h20=$(grep '^  Added' leap.log | awk '{print $2}') 

#The number of solvent molecules in 1 L
n_kcl_in_one_liter=$(awk "BEGIN {print $n_kcl * $n_a; exit}") 

#The total number of solvent molecules
n0=$(awk "BEGIN {printf $n_kcl_in_one_liter / $n_h20_in_one_liter * $n_h20; exit}") 
n0=${n0%.*}

# The system's charge
q=$(grep -i "Total solute charge" leap.log | awk '{print $4}')

# final number of K+ and Cl- to neutralize system and to match the target salt concentration (SLTCAP)
n_k=$(echo "scale=10; $n0 * sqrt(1 + 1 / (2 * $n0)^2) - $q / 2" | bc)
n_cl=$(echo "scale=10; $n0 * sqrt(1 + 1 / (2 * $n0)^2) + $q / 2" | bc -l)
# rounding Numbers
n_k=$(printf '%1.0f' $n_k)
n_cl=$(printf '%1.0f' $n_cl)

echo $n_k, $n_cl #The number of K and Cl molecules

# Editing script

cp scripts/creating_box/wo_ions.rc scripts/creating_box/w_ions.rc

sed -i "s/addions mol K+ 0/addions mol K+ ${n_k}/g" scripts/creating_box/w_ions.rc
sed -i "s/addions mol Cl- 0/addions mol Cl- ${n_cl}/g" scripts/creating_box/w_ions.rc
sed -i "s/box_wo_kcl/box_kcl/g" scripts/creating_box/w_ions.rc
sed -i "s/draft_box/solvated_box/g" scripts/creating_box/w_ions.rc

# Starting simulation with new ion settings
cd force_field
mkdir box_kcl
cd ..
rm leap.log

module load amber24


tleap -s -f scripts/creating_box/w_ions.rc # -s: programm will write a .log file; -f: file path


## 2.0. Energy minimization and System Equillibration
To prevent formation of unusual high in energy structural patterns (e.g. close overlapse of Lennard-Jhonnes atomes) and stabilize the molecule, energy minimization should be performed. In Amber there are two minimization modules: sander and pmemd (see 21. sander in Amber2024) they both require special input file .in.

The full process includes 3 main steps*:
- Minimization
- Heating in a box with a constant value in a thermostat
- Relaxation (equillibration) at constnat temperature and constant pressure

Heating proccess depends on thermostat choice, which is defined in ntt variable in .in. Here we will be using ntt=1 - weak-coupling 
algorithm, it rescales total kinetic energy of the system, poured in a heat bath, to match the desired temperature. Note that this approach does not count local thermal changes. Weak-coupling here means that interaction between the structure and the hating bath is being ignored.

For big systems like nucleosomes it may be necessary to firstly restrain the molecule (Note the apex of min_1_draft.in and ntr value), although in this tutorial we will remove the restrain.

Let's have a look on .in and .sh files 

*Breefly (https://ambermd.org/tutorials/basic/tutorial14/index.php)

In [ ]:
%%bash
cd scripts/min_eq
cat min_1_draft.in

The .in file should be edited before run, i.e. restrains should be deleted 

In [ ]:
%%bash
cat scripts/min_eq/min_1.in

In [ ]:
%%bash

cat scripts/min_eq/newton_job_min_1.sh #the script for minimization, in your case it might be necessary to insert GPU, check __FILL__ .sh
#creating folders for output
mkdir minimization
mkdir minimization/min_1

## 2.1. Initial minimization run 
Let's run the first minimization! 

In [ ]:
%%bash
#chmod +x scripts/min_eq/newton_job_min_1.sh - to make script executive
./scripts/min_eq/newton_job_min_1.sh

## 2.1. Minimization, Heating, and Equilibration
Let's have a look on a script, which allows us to run minimization and further heating and equilibration all at once, and run it

In [ ]:
%%bash
#creating folders for output
mkdir minimization/min_2
mkdir minimization/heat
mkdir minimization/equil
cat scripts/min_eq/newton_job_min_eq.sh

In [ ]:
%%bash 
#chmod +x scripts/min_eq/newton_job_min_eq.sh
./scripts/min_eq/newton_job_min_eq.sh

## 2.2. Visualization
Let's visualizate the current progress and detect differences between three obtained structures. Firstly rewrite final coordinate file into new .pdb with a use of cpptraj.

*Note that output file does not contain information about molecule types (i.g. DNA or protein), that's why you should additionally edit it. See the second cell.

In [ ]:
!cat scripts/trajectory_processing/pdb_converter.in
!module load amber24
!mkdir trajectory

#making .pdb files from .ncrst
for folder in ['min_2',
              'heat',
              'equil']:
    # !rm -rf trajectory/"$folder"
    !mkdir trajectory/"$folder"
    !cp scripts/trajectory_processing/pdb_converter.in scripts/trajectory_processing/"$folder".in
    !cp force_field/box_kcl/solvated_box.prmtop trajectory/"$folder"/"$folder".prmtop
    !sed -i "s#INITIAL_FILENAME#minimization/$folder/$folder#g" scripts/trajectory_processing/"$folder".in 
    !sed -i "s#OUTPUT#trajectory/$folder/$folder#g" scripts/trajectory_processing/"$folder".in 
    !cpptraj -i scripts/trajectory_processing/"$folder".in > scripts/trajectory_processing/cpptraj_"$folder".out 

In [ ]:
import MDAnalysis as mda
import numpy as np 

#editing the output
for folder in ['min_2',
              'heat',
              'equil']:
    #!rm minimization/"$folder"/sys_"$folder".pdb
    !mdconvert minimization/"$folder"/"$folder".pdb -o minimization/"$folder"/sys_"$folder".pdb
    
    #note that segid's indexes A,B,C etc. are placed in an alphabet order and may differ from the initial placement
    
    u = mda.Universe(f'minimization/{folder}/sys_{folder}.pdb')
    
    #Rescaling of residues' indexes: you can see, that in initial .pdb similar residues have the same index
    for seg in u.segments:
        if seg.segid == 'A' or seg.segid == 'B':
            seg.residues.resids  = np.arange(-(len(seg.residues.resids)//2), len(seg.residues.resids)//2+1) #step ONLY for nucleosomes
        else:
            seg.residues.resids  = np.arange(1, len(seg.residues.resids)+1) #step for proteins or DNA

In [ ]:
import nglview as nv
min2=nv.show_structure_file('minimization/min_2/sys_min_2.pdb')
heat=nv.show_structure_file('minimization/heat/sys_heat.pdb')
equil=nv.show_structure_file('minimization/equil/sys_equil.pdb')
# a.add_representation(selection='water',repr_type='spacefill',color='red') #molecule in the box
min2.render_image()
heat.render_image()
equil.render_image()
min2

In [ ]:
heat

In [ ]:
equil 

## 3.0. Trajectory
Now let's process a trajectory to see the molecule moving!

In [ ]:
# Create XTC from NC files 
## Create run file
!module load amber24

for folder in ['heat',
              'equil']: 
    !echo "parm force_field/box_kcl/solvated_box.prmtop" > trajectory/cpptraj_in.in
    !echo "trajin minimization/{folder}/{folder}.nc" >> trajectory/cpptraj_in.in
    !echo "trajout trajectory/{folder}/{folder}.nc" >> trajectory/cpptraj_in.in
    !echo "trajout trajectory/{folder}/{folder}.xtc" >> trajectory/cpptraj_in.in
    !echo "strip :WAT,K+,Na+,Cl- parmout trajectory/{folder}/{folder}.prmtop" >> trajectory/cpptraj_in.in
    !echo "autoimage" >> trajectory/cpptraj_in.in
    !echo "run" >> trajectory/cpptraj_in.in
    !cpptraj -i trajectory/cpptraj_in.in
 

In [ ]:
import MDAnalysis as mda
import nglview as nv

#Note that starting .pdb file was obtained on a previous step

pdb = mda.Universe('minimization/min_2/sys_min_2.pdb', 'trajectory/heat/heat.xtc')
nv.show_mdanalysis(pdb)

In [ ]:
pdb = mda.Universe('minimization/heat/sys_heat.pdb', 'trajectory/equil/equil.xtc')
nv.show_mdanalysis(pdb)

## 4.0 Energy
Let's obtain energy curves for heating and equilibration. Reminder: in .in script file for heating we 

In [ ]:
#Making an initial .in file for cpptraj

!module load amber24

for folder in ['heat',
              'equil']:
    !echo "parm trajectory/{folder}/{folder}.prmtop" > trajectory/cpptraj_in_energy_{folder}.in 
    !echo "reference minimization/{folder}/sys_{folder}.pdb" >> trajectory/cpptraj_in_energy_{folder}.in
    !echo "trajin trajectory/{folder}/{folder}.nc" >> trajectory/cpptraj_in_energy_{folder}.in
    !echo "energy out trajectory/ene_{folder}.txt" >> trajectory/cpptraj_in_energy_{folder}.in
    !cpptraj -i trajectory/cpptraj_in_energy_{folder}.in 

In [ ]:
import pandas as pd 

#Visualisation of energy curves
#d = pd.DataFrame(np.loadtxt(f'trajectory/ene_equil.txt'), columns = 'Frame   ENE_00002[bond] ENE_00002[angle] ENE_00002[dih] ENE_00002[vdw14] ENE_00002[elec14] ENE_00002[vdw] ENE_00002[elec] ENE_00002[kinetic] ENE_00002[total]'.split())

d = pd.DataFrame(np.loadtxt(f'trajectory/ene_heat.txt'), columns = 'Frame   ENE_00002[bond] ENE_00002[angle] ENE_00002[dih] ENE_00002[vdw14] ENE_00002[elec14] ENE_00002[vdw] ENE_00002[elec] ENE_00002[kinetic] ENE_00002[total]'.split())
for c in 'ENE_00002[bond] ENE_00002[angle] ENE_00002[dih] ENE_00002[vdw14] ENE_00002[elec14] ENE_00002[vdw] ENE_00002[elec] ENE_00002[kinetic] ENE_00002[total]'.split():
    d.loc[:,['Frame',f'{c}']].plot(x = 'Frame', title = 'heat')
  


d = pd.DataFrame(np.loadtxt(f'trajectory/ene_equil.txt'), columns = 'Frame   ENE_00002[bond] ENE_00002[angle] ENE_00002[dih] ENE_00002[vdw14] ENE_00002[elec14] ENE_00002[vdw] ENE_00002[elec] ENE_00002[total]'.split())

for c in 'ENE_00002[bond] ENE_00002[angle] ENE_00002[dih] ENE_00002[vdw14] ENE_00002[elec14] ENE_00002[vdw] ENE_00002[elec] ENE_00002[total]'.split():
    d.loc[:,['Frame',f'{c}']].plot(x = 'Frame', title = 'equil')

## 5.0. Accuracy of MD
To confirm the accuracy of calculations, made previousely, several checks are required:
- Profiles of main MD parameters (temperature, pressure, energies)
- Distances between unit cells during simulation in order

MD parameters' data can be collected with process_mdout.perl command. It returns a full information about the system (.TEMP, .ETOT, .DENCITY, etc.)

Distances between images can be calculated with cpptraj module. Minimage calculates distances between selceted atoms/residues/molecules of two neigbouhring periodic images by default(https://amberhub.chpc.utah.edu/minimage/, https://amberhub.chpc.utah.edu/atom-mask-selection-syntax/). In this tutorial we will calculate distances only for equil.nc, in time-saving purposes.

### 5.1. process_mdout.perl

In [ ]:
%%bash
#collecting outputs
module load amber24

mkdir output
cd output

process_mdout.perl ../minimization/min_2/min_2.out ../minimization/heat/heat.out ../minimization/equil/equil.out

In [ ]:
import numpy as np
import pandas as pd 

#Temperature Curve
d = pd.DataFrame(np.loadtxt('output/summary.TEMP'), columns = ['time (ps)', 'Temperature (K)'])
d.plot(x = 'time (ps)', y = 'Temperature (K)', title='MD Temperature').set_ylabel('Temperature (K)')

#Energy curves
d1 = pd.DataFrame(np.loadtxt('output/summary.ETOT'), columns = ['time (ps)', 'ETOT']) #Total Energy
d2 = pd.DataFrame(np.loadtxt('output/summary.EPTOT'), columns = ['time (ps)', 'EPTOT']) #Total Potintial Energy
d3 = pd.DataFrame(np.loadtxt('output/summary.EKTOT'), columns = ['time (ps)', 'EKTOT']) #Total Kinetic Energy
d1['EPTOT'] = d2['EPTOT']
d1['EKTOT'] = d3['EKTOT']
d1.plot(x = 'time (ps)', y = ['ETOT','EPTOT','EKTOT'], title='MD Energy', ylim=(-1500000, 250000)).set_ylabel('Energy (kcal/mol)')

#You can visualize each curve separatelly
# d1.plot(x = 'time (ps)', y = ['EKTOT'], title='MD Energy').set_ylabel('Energy (kcal/mol)')
# d1.plot(x = 'time (ps)', y = ['EPTOT'], title='MD Energy').set_ylabel('Energy (kcal/mol)')
# d1.plot(x = 'time (ps)', y = ['ETOT'], title='MD Energy').set_ylabel('Energy (kcal/mol)')

#Pressure Curve
d = pd.DataFrame(np.loadtxt('output/summary.PRES'), columns = ['time (ps)', 'Pressure (bar)'])
d.plot(x = 'time (ps)', y = 'Pressure (bar)', title='MD Pressure').set_ylabel('Pressure (bar)')


### 5.2. minimage
Note the 'autoimage' in .in file, it centeres trajectory with .promtop

In [ ]:
!cat scripts/accuracy/minimage_script.in

In [ ]:
%%bash
module load amber24
cpptraj -i scripts/accuracy/minimage_script.in -o scripts/accuracy/minimage.out

In [ ]:
import numpy as np
import pandas as pd

#Distances between replicas
d = np.loadtxt('output/output_min_distance.xvg')
data = pd.DataFrame(d, columns = ['Frame', 'Dist', 'A1', 'A2'])
data.plot.line(x='Frame', y='Dist', title = 'Distance between replicas',grid=True).set_ylabel('Distance (A)')

#Distance deviations between replicas
mean = np.mean(d[:,1])
s = np.std(d[:,1])
for i in range(len(d)):
    d[i,1] = abs(d[i,1] - mean) 
data = pd.DataFrame(d, columns = ['Frame', 'Dist', 'A1', 'A2'])
data['std'] = s
data.plot.line(x='Frame', y=['Dist', 'std'], title = 'Distance deviation between replicas',grid=True).set_ylabel('Distance (A)')


### 0.0.0. Clear Your Workspace

In [ ]:
!rm -rf prep
!rm scripts/creating_box/w_ions.rc
!rm leap.log
!rm -rf force_field
!rm -rf trajectory
!rm -rf output
for folder in ['min_1'
              'min_2',
              'heat',
              'equil']:
    !rm -rf minimization/"$folder"
    !mkdir minimization/"$folder""
